"Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe"

Imports

In [172]:
#imports
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

Get the data from the page

In [178]:
html_doc = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
#a soup object of the whole page
soup = BeautifulSoup(html_doc, 'html.parser')

#just the table we're interested
table = soup.find('table', class_='wikitable sortable')

#parse the table entries into a list
wikis = []

rows = table.find_all('tr')
for row in rows:
    cells = row.find_all('td')
    if len(cells)==3:
        postalCode = cells[0].find(text=True)
        borough = cells[1].find(text=True)
        neigh = cells[2].find(text=True)
        
        #Ignore cells with a borough that is Not assigned
        if "Not assigned" not in borough:
            #append to list
            wikis.append([postalCode, borough, neigh])
        
#print(wikis)
        

Transform the data into a pandas dataframe and prepare it

In [226]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# create the dataframe
dfWikis = pd.DataFrame(wikis, columns=column_names)

#combine neighborhoods in the same boroughs
dfPostalCode = dfWikis.groupby(['PostalCode','Borough'], sort = False).agg(lambda x: ','.join(x))

dfPostalCode = dfPostalCode.reset_index()

#if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
dfPostalCode.where(dfPostalCode['PostalCode'] == 'Not assigned\n', dfPostalCode['Borough'], inplace=True, axis=0)
#dfPostalCode = dfGrouped


Use the .shape method to print the number of rows of the dataframe

In [228]:
#dfPostalCodes.shape
dfPostalCode.shape

(103, 3)